In [1]:
!pip install transformers
import transformers
import torch
import pandas as pd
import numpy as np
from torch.utils.data import Dataset,DataLoader
from transformers import AutoTokenizer,AutoModel
pairdf=pd.read_csv("/content/drive/MyDrive/PDHealthCareProject/bioBERTDocClf/paridf.csv",index_col=0)
pairdf.head()

     |████████████████████████████████| 2.8 MB 5.3 MB/s 
     |████████████████████████████████| 3.3 MB 38.1 MB/s 
     |████████████████████████████████| 895 kB 49.3 MB/s 
     |████████████████████████████████| 636 kB 49.3 MB/s 
     |████████████████████████████████| 50 kB 6.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


,text1,text2,is_same
0,"PREOPERATIVE DIAGNOSES:,1. Request for cosmet...","PREOPERATIVE DIAGNOSIS:, Ageing face.,POSTOPE...",1
1,"PREOPERATIVE DIAGNOSIS: , Right lateral base o...","CHIEF COMPLAINT: , Newly diagnosed T-cell lymp...",1
2,"DIAGNOSIS: , Bilateral hypomastia.,NAME OF OPE...","DIAGNOSIS:, Stasis ulcers of the lower extrem...",1
3,"REASON FOR VISIT:, Weight loss evaluation.,HI...","SUBJECTIVE:, Mom brings patient in today beca...",1
4,"PREOPERATIVE DIAGNOSIS:, Desire for sterility...","Sample Address,RE: Sample Patient,Wife's name...",1


In [ ]:
10 docs--
heart, cancer, bypass surgey



In [2]:
class PairDocData(Dataset):
    def __init__(self,dataframe,tokenizer,max_len=512):
        self.len=len(dataframe)
        self.max_len=max_len
        self.tokenizer=tokenizer
        self.data=dataframe

    def __getitem__(self,index):
        text1=str(self.data['text1'][index])
        text2=str(self.data['text2'][index])
        label=self.data['is_same'][index]
        text1=' '.join(text1.split())
        text2=' '.join(text2.split())
        inputs1=self.tokenizer.encode_plus(
            text1,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True,
            truncation=True
        )
        inputs2=self.tokenizer.encode_plus(
            text2,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True,
            truncation=True
        )
        ids1,mask1 = inputs1['input_ids'],inputs1['attention_mask'] 
        ids1=torch.tensor(ids1, dtype=torch.long)
        mask1=torch.tensor(mask1,dtype=torch.long)
        ids2,mask2 = inputs2['input_ids'],inputs2['attention_mask'] 
        ids2=torch.tensor(ids2, dtype=torch.long)
        mask2=torch.tensor(mask2,dtype=torch.long)

        return {
            'ids1':ids1,'mask1':mask1,'ids2':ids2,'mask2':mask2,
            'targets': torch.tensor(label, dtype=torch.long)
        }     

    def __len__(self):
        return self.len

In [3]:
train_size = 0.8
train_dataset=pairdf.sample(frac=train_size,random_state=200)
test_dataset=pairdf.drop(train_dataset.index).reset_index(drop=True)
train_dataset = train_dataset.reset_index(drop=True)
tokenizer = AutoTokenizer.from_pretrained('dmis-lab/biobert-base-cased-v1.1')

training_set=PairDocData(train_dataset,tokenizer)
testing_set=PairDocData(test_dataset,tokenizer)

train_params = {'batch_size': 4,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': 4,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

Downloading:   0%|          | 0.00/313 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

In [4]:
class BertFewShot(torch.nn.Module):
    def __init__(self):
        super(BertFewShot,self).__init__()
        self.bert=AutoModel.from_pretrained("dmis-lab/biobert-base-cased-v1.1")
        self.pre_clf=torch.nn.Linear(768,768)
        self.dropout=torch.nn.Dropout(0.3)
        self.dim_100=torch.nn.Linear(768,100)
        self.last_layer=torch.nn.Linear(100,1)
        self.sigmoid=torch.nn.Sigmoid()

    def get_100(self,ids1,mask1):
        output1=self.bert(input_ids=ids1,attention_mask=mask1)
        hidden_state = output1[0]
        pooler = hidden_state[:, 0]
        pooler=self.pre_clf(pooler)
        pooler=torch.nn.ReLU()(pooler)
        pooler=self.dropout(pooler)
        output=self.dim_100(pooler)
        return output


    def forward(self,ids1,mask1,ids2,mask2):
        x1=self.get_100(ids1,mask1)
        x2=self.get_100(ids2,mask2)
        x=torch.abs(x1-x2)
        x=self.last_layer(x)
        x=self.sigmoid(x)
        return x
model=BertFewShot()
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

Downloading:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of the model checkpoint at dmis-lab/biobert-base-cased-v1.1 were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertFewShot(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)


In [5]:
loss_function = torch.nn.BCELoss()
optimizer = torch.optim.Adam(params =  model.parameters(), lr=1e-05)

In [6]:
def train(epoch):
    model.train()
    n_steps=0
    tr_loss=0
    for _,data in enumerate(training_loader,0):
        # print(n_steps)
        ids1 = data['ids1'].to(device, dtype = torch.long)
        mask1=data['mask1'].to(device,dtype=torch.long)
        ids2=data['ids2'].to(device,dtype=torch.long)
        mask2=data['mask2'].to(device,dtype=torch.long)
        targets=data['targets'].to(device,dtype=torch.float)
        outputs=model(ids1,mask1,ids2,mask2)
        # print(outputs)
        # print(targets)
        loss=loss_function(outputs.squeeze(1),targets)
        # print("loss:",loss)
        tr_loss+=loss.item()

        n_steps+=1
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if(n_steps%50==0):
            print("loss after steps:",n_steps,"is",tr_loss/n_steps)

    print("loss After ",epoch,"epochs:",tr_loss/n_steps)



In [7]:
for i in range(5):
    train(i)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2204: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


loss after steps: 50 is 0.6892541921138764
loss after steps: 100 is 0.6873280423879623
loss after steps: 150 is 0.6820369688669841
loss after steps: 200 is 0.6716880950331688
loss after steps: 250 is 0.649545508980751
loss after steps: 300 is 0.6302162741621335
loss after steps: 350 is 0.6055631265044212
loss after steps: 400 is 0.5834679260477423
loss after steps: 450 is 0.5627342152843873
loss after steps: 500 is 0.5470012798085808
loss after steps: 550 is 0.5346964717927304
loss after steps: 600 is 0.5217677707659701
loss after steps: 650 is 0.5099611117977363
loss after steps: 700 is 0.49990082224830984
loss after steps: 750 is 0.4902047457396984
loss after steps: 800 is 0.4878045308962464
loss after steps: 850 is 0.4792497013706495
loss after steps: 900 is 0.4736785965681904
loss after steps: 950 is 0.47005192160410314
loss after steps: 1000 is 0.4660961245447397
loss after steps: 1050 is 0.4617132300422305
loss after steps: 1100 is 0.45795697437937966
loss after steps: 1150 is 0.

KeyboardInterrupt: ignored

In [8]:
torch.save(model,"/content/drive/MyDrive/PDHealthCareProject/bioBERTDocClf/fewshotmodel.pth")